In [8]:
import pandas as pd 
import requests as rq 
from bs4 import BeautifulSoup



In [9]:
raw_list = rq.get('https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/')
raw_list.status_code
print(raw_list)
list_page = raw_list.content

<Response [200]>


In [10]:
#soup
list_bsp = BeautifulSoup(list_page, 'html.parser')
list_bsp

<!DOCTYPE html>

<!--[if lt IE 7]><html class="no-js ie ie6 lt-ie9 lt-ie8 lt-ie7" lang="fr-FR"> <![endif]--><!--[if IE 7]><html class="no-js ie ie7 lt-ie9 lt-ie8" lang="fr-FR"> <![endif]--><!--[if IE 8]><html class="no-js ie ie8 lt-ie9" lang="fr-FR"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="fr-FR">
<!--<![endif]--><head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link charset="utf-8" href="https://www.assureurs.pro/wp-content/themes/lightwords-v3/style_18.css?20200309-v1" media="screen" rel="stylesheet" type="text/css"/>
<link href="https://www.assureurs.pro/xmlrpc.php" rel="pingback"/>
<link href="https://www.assureurs.pro/wp-content/uploads/2022/08/favicon-assureur-16x0-c-default.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="https://www.assureurs.pro/wp-content/uploa

SCRAPING URLS SUBPAGES NORD

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get('https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/nord/')

# Find all "a" elements on the page
list_links = driver.find_elements(By.TAG_NAME, 'a')

# Create an empty list to store the href attributes
list_url = []

# Iterate through each anchor element

for link in list_links:
    # Get the href attribute and append it to the list_url
    href = link.get_attribute('href')
    if href == None or link == None :
        continue
    else :
        list_url.append(href)


driver.quit()

# Print or use the list_url as needed
print(list_url)




['https://www.assureurs.pro/', 'https://www.assureurs.pro/', 'https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/nord/#', 'https://www.assureurs.pro/banque/', 'https://www.assureurs.pro/assurance/', 'https://www.assureurs.pro/epargne-et-investissement/', 'https://www.assureurs.pro/impots/', 'https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/', 'https://www.assureurs.pro/demande-de-devis/', 'https://www.assureurs.pro/guides-pour-les-pros/', 'https://www.assureurs.pro/lp-digitalisez-votre-entreprise/', 'https://www.assureurs.pro/', 'https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/', 'https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/', 'https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/nord/#', 'https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/nord/#', 'http://leafletjs.com/', 'https://www.openstree

In [12]:
list_url_pd = pd.DataFrame({'URL': list_url})
# Save the DataFrame to a CSV file
csv_filename = 'output_urls_AllNord.csv'
list_url_pd.to_csv(csv_filename, index=True)


SCRAPINGS SUBPAGES

In [13]:
import re

def extract_insurers_lists_as_dataframe(url):
    try:
        #only apply this function to url who contains adresses 
        #https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/nord/
        #optionally followed by all of a literal /, optionally followed by a literal home, optionally followed by a literal ? and any number of other characters.
        patternUrl = r"^https:\/\/www\.assureurs\.pro\/entreprise_localisation\/toutes-les-regions\/nord-pas-de-calais\/nord\/([^']+)\/"
        matchUrl = re.search(patternUrl, url)
        if matchUrl:
            page = rq.get(url)
            page.raise_for_status()
            soup = BeautifulSoup(page.content, 'html.parser')

            #extract artcile within targetted url
            
            elements = soup.find_all('article', onclick=True)
            print(elements)
            

            # Extract the URL using regular expression
            if elements:
                for element in elements:
                    onclick_value = element['onclick'] #following BS list created including onclick
                    # Use re.search to find the pattern in the onclick_value
                    pattern = r"window\.location\.href\s*=\s*'([^']+)'" 
                    match = re.search(pattern, onclick_value)
                    
                    
                    if match:
                        url2 = match.group(1) #(1)?
                        print(f'Extracted URL: {url2}')
                    # create a dic with basic info (.text.strip())

                        
                        #info = {
                        #'onclick': url2,
                        #'name': [item.text.strip() if item else "Aucun titre" for item in soup.select('.tile.tile-entreprise a')],
                        #'address': [item.text.strip() if item else "Aucune description" for item in soup.select('.tile.tile-entreprise .tile__body')],
                        #}

                        #'onclick': url2,
                        #'name': soup.select('.tile.tile-entreprise a') if soup.select('.tile.tile-entreprise a') else "Aucun titre",
                        #'adress': soup.select('.tile.tile-entreprise .tile__body') if soup.select('.tile.tile-entreprise .tile__body') else "Aucune adresse"
                    
                    #data.append(info)

                    # Select all elements "div" inside "articles" that match the specified class within information
                        tiles = soup.select('.tile.tile-entreprise')

                        # Create a list to store dictionaries
                        text_div_list = []

                    # Loop through each tile element and extract data
                    for index, tile in enumerate(tiles):
                        onclick_value = url2
                        name_values = [a.text for a in tile.select('a')] or "Aucun titre"
                        address_values = [body.text for body in tile.select('.tile__body')] or "Aucune adresse"

                        # Fill dictionary for the current set of data
                        data_dict = {
                            'url': onclick_value,
                            'name': name_values,
                            'address': address_values
                        }

                        # Append the dictionary to the list
                        text_div_list.append(data_dict)

                        
                        # Display the resulting list of dictionaries
                        for data_dict in text_div_list:
                            print(data_dict)

                    else: #(if no match)
                        print('No URL found in onclick attribute.')
                        continue
                    
                df = pd.DataFrame(text_div_list) 
                return df  
            else:   
                print('Element with onclick attribute not found.')
        else: 
            print('No matched URL')
    except rq.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de la page : {e}")
        #return pd.DataFrame()

# path CSV file including urls
file_path = 'C:\\Users\\BGE\\ownCloud\\Arnaud\\DEVELOPPEMENT\\ACTIVITE\\Sponso owncloud perso AS\\Scraping\\Assu_Nord_012024\\output_urls_AllNord.csv'

# Read CSV file including urls
urls_df = pd.read_csv(file_path, header=None)

# Apply function on every URL and concatenate résults
all_data_frames = [extract_insurers_lists_as_dataframe(url) for url in urls_df[1]]
print(all_data_frames)

result_df = pd.concat(all_data_frames, ignore_index=False)

# Afficher ou sauvegarder le DataFrame résultant
result_df
# result_df.to_csv('chemin_pour_sauvegarder.csv')  # Décommenter pour sauvegarder dans un fichier CSV


No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
No matched URL
[<article class="tile tile-entreprise flex-xs-6 flex-sm-6 flex-md-4 flex-lg-3" onclick="window.location.href = 'https://www.assureurs.pro/entreprise/courtage-assurances-mutuelles-finance-820954279/';"><div class="tile__head"><div class="tile__hero"><div class="h3 tile__title text-center"><a href="https://www.assureurs.pro/entreprise/courtage-assurances-mutuelles-finance-820954279/">Société COURTAGE ASSURANCES MUTUELLES FINANCE (SARL)</a></div></div></div>
<div class="tile__body">
<div>20 Rte De Le Quesnoy</div>
<div>59570 Bermeries</div>
</div></article>, <article class="tile tile-entrepr

,url,name,address
0,https://www.assureurs.pro/entreprise/davran-ka...,[Société COURTAGE ASSURANCES MUTUELLES FINANCE...,[\n20 Rte De Le Quesnoy\n59570 Bermeries\n]
1,https://www.assureurs.pro/entreprise/davran-ka...,[Société SOFCADE (SAS)],[\n21 Rue Royale\n59800 Lille\n]
2,https://www.assureurs.pro/entreprise/davran-ka...,[Entreprise LEFEBVRE ALAIN],[\n28 Rue Des Digues\n59360 Le-Cateau-Cambresi...
3,https://www.assureurs.pro/entreprise/davran-ka...,[Société MUTUELLE DE RETRAITE DES A.C DU NORD],[\n13 Rue Jacquemars Gielee\n59800 Lille\n]
4,https://www.assureurs.pro/entreprise/davran-ka...,[Société PETIT SABRINA],[\n18 Rue Diderot\n59790 Ronchin\n]
...,...,...,...
3,https://www.assureurs.pro/entreprise_localisat...,[Entreprise PREVISANTE (SARL)],[\n14 Rue Wilson\n59490 Somain\n]
4,https://www.assureurs.pro/entreprise_localisat...,[Entreprise SULPIZI SANDRA],[\n22 Rue Wilson\n59490 Somain\n]
5,https://www.assureurs.pro/entreprise_localisat...,[Société LALOYAUX MATTHIEU],[\n27 Rue Suzanne Lanoy\n59490 Somain\n]
6,https://www.assureurs.pro/entreprise_localisat...,[Société BANKIEWIEZ VIRGINIE],[\n10 Rue Edwin Labendzki\n59490 Somain\n]


Clean datas and split columns

In [14]:
#cleaning
insurers_list = result_df.applymap(lambda x: ''.join(x).strip())

#splitting 
# Create 'insurers_list_addresses' DataFrame
insurers_list_addresses = pd.DataFrame()
insurers_list_addresses[['street', 'zipcode and city']] = insurers_list['address'].str.split('\n', expand=True)

insurers_list_cities= pd.DataFrame()
insurers_list_cities[['zipcode','city']] = insurers_list_addresses['zipcode and city'].str.split(' ', expand=True)
insurers_list_cities

#concatening
insurers= pd.DataFrame()
insurers = [insurers_list, insurers_list_addresses, insurers_list_cities]
insurers_list_vf = pd.concat(insurers, axis=1)


In [15]:
# Set the index with the number of the line as a unique identifier
insurers_list_vf.index = range(1, len(insurers_list_vf) + 1)
insurers_list_vf

,url,name,address,street,zipcode and city,zipcode,city
1,https://www.assureurs.pro/entreprise/davran-ka...,Société COURTAGE ASSURANCES MUTUELLES FINANCE ...,20 Rte De Le Quesnoy\n59570 Bermeries,20 Rte De Le Quesnoy,59570 Bermeries,59570,Bermeries
2,https://www.assureurs.pro/entreprise/davran-ka...,Société SOFCADE (SAS),21 Rue Royale\n59800 Lille,21 Rue Royale,59800 Lille,59800,Lille
3,https://www.assureurs.pro/entreprise/davran-ka...,Entreprise LEFEBVRE ALAIN,28 Rue Des Digues\n59360 Le-Cateau-Cambresis,28 Rue Des Digues,59360 Le-Cateau-Cambresis,59360,Le-Cateau-Cambresis
4,https://www.assureurs.pro/entreprise/davran-ka...,Société MUTUELLE DE RETRAITE DES A.C DU NORD,13 Rue Jacquemars Gielee\n59800 Lille,13 Rue Jacquemars Gielee,59800 Lille,59800,Lille
5,https://www.assureurs.pro/entreprise/davran-ka...,Société PETIT SABRINA,18 Rue Diderot\n59790 Ronchin,18 Rue Diderot,59790 Ronchin,59790,Ronchin
...,...,...,...,...,...,...,...
184,https://www.assureurs.pro/entreprise_localisat...,Entreprise PREVISANTE (SARL),14 Rue Wilson\n59490 Somain,14 Rue Wilson,59490 Somain,59490,Somain
185,https://www.assureurs.pro/entreprise_localisat...,Entreprise SULPIZI SANDRA,22 Rue Wilson\n59490 Somain,22 Rue Wilson,59490 Somain,59490,Somain
186,https://www.assureurs.pro/entreprise_localisat...,Société LALOYAUX MATTHIEU,27 Rue Suzanne Lanoy\n59490 Somain,27 Rue Suzanne Lanoy,59490 Somain,59490,Somain
187,https://www.assureurs.pro/entreprise_localisat...,Société BANKIEWIEZ VIRGINIE,10 Rue Edwin Labendzki\n59490 Somain,10 Rue Edwin Labendzki,59490 Somain,59490,Somain


In [16]:
insurers_list_vf = insurers_list_vf[:][['url', 'name', 'street', 'zipcode', 'city']]
insurers_list_vf = insurers_list_vf.rename_axis('ID')
insurers_list_vf.sort_values('city')

,url,name,street,zipcode,city
ID,,,,,
171,https://www.assureurs.pro/entreprise_localisat...,Société BLANQUET FLORIAN,201 Che Rural Feuter Dreve,59270,Bailleul
170,https://www.assureurs.pro/entreprise_localisat...,Société REFERENCE COURTAGE (SARL),13 Rue Saint Jacques,59270,Bailleul
169,https://www.assureurs.pro/entreprise_localisat...,Entreprise DELAFOSSE MARIE-YVONNE,105 Rue D'ypres,59270,Bailleul
168,https://www.assureurs.pro/entreprise_localisat...,Entreprise DELESALLE GHISLAIN,39 Gpl Gde Pce Charles De Gaulle,59270,Bailleul
167,https://www.assureurs.pro/entreprise_localisat...,Société ROUSSET PHILIPPE,251 Rue De Lille,59270,Bailleul
...,...,...,...,...,...
156,https://www.assureurs.pro/entreprise_localisat...,Autres assureurs à Mouvaux,Aucune adresse,None,None
164,https://www.assureurs.pro/entreprise_localisat...,Autres assureurs à Lambersart,Aucune adresse,None,None
172,https://www.assureurs.pro/entreprise_localisat...,Autres assureurs à Bailleul,Aucune adresse,None,None


In [17]:
#erase "None" address 
import numpy as np
insurers_list_vf['street'].replace('Aucune adresse', np.nan)
insurers_list_vf.dropna()

,url,name,street,zipcode,city
ID,,,,,
1,https://www.assureurs.pro/entreprise/davran-ka...,Société COURTAGE ASSURANCES MUTUELLES FINANCE ...,20 Rte De Le Quesnoy,59570,Bermeries
2,https://www.assureurs.pro/entreprise/davran-ka...,Société SOFCADE (SAS),21 Rue Royale,59800,Lille
3,https://www.assureurs.pro/entreprise/davran-ka...,Entreprise LEFEBVRE ALAIN,28 Rue Des Digues,59360,Le-Cateau-Cambresis
4,https://www.assureurs.pro/entreprise/davran-ka...,Société MUTUELLE DE RETRAITE DES A.C DU NORD,13 Rue Jacquemars Gielee,59800,Lille
5,https://www.assureurs.pro/entreprise/davran-ka...,Société PETIT SABRINA,18 Rue Diderot,59790,Ronchin
...,...,...,...,...,...
183,https://www.assureurs.pro/entreprise_localisat...,Société FORTE JULIEN,22 Rue Wilson,59490,Somain
184,https://www.assureurs.pro/entreprise_localisat...,Entreprise PREVISANTE (SARL),14 Rue Wilson,59490,Somain
185,https://www.assureurs.pro/entreprise_localisat...,Entreprise SULPIZI SANDRA,22 Rue Wilson,59490,Somain


In [18]:
insurers_list_vf.groupby('city').size()

city
Bailleul                 7
Bermeries                1
Cambrai                  7
Caudry                   7
Croix                    7
Douai                    8
Dunkerque                7
Hazebrouck               7
La-Madeleine             7
Lambersart               7
Le-Cateau-Cambresis      1
Lille                   10
Marcq-en-Baroeul         8
Maubeuge                 7
Mons-en-Baroeul          7
Mouvaux                  7
Ronchin                  1
Roubaix                  7
Saint-Amand-les-Eaux     7
Sepmeries                1
Sin-le-Noble             1
Somain                   7
Tourcoing                7
Valenciennes             8
Villeneuve-d'Ascq        7
Wambrechies              7
Wasquehal                7
Wattrelos                1
dtype: int64

In [19]:
result_df.applymap(lambda x: str(x).strip())
csv_filename = 'outputs_lists_insurers_AllNord.csv'
insurers_list_vf.to_csv(csv_filename)


PermissionError: [Errno 13] Permission denied: 'outputs_lists_insurers_AllNord.csv'

SELENIUM / WEBDRIVER - send form response

In [20]:
#webdriverwait and Explicit COnditions 

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [21]:


# Set up the WebDriver (adjust the path based on your browser)
from selenium.webdriver.chrome.service import Service
service = Service(executable_path='C:\\Users\\BGE\\chromedriver_win32\\chromedriver.exe')

#instance webdriver

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

#requests looping list of pages including forms
urls_forms = list(insurers_list_vf['url'])
for url_forms in urls_forms:
    driver.get(url_forms)

    # Use WebDriverWait to wait for the form elements to be present 
    try:
        element = WebDriverWait(driver, 10).until(
            # box welcoming text to express the need
            EC.presence_of_element_located((By.NAME, 'input_9')),
            #firstname
            EC.presence_of_element_located((By.NAME, 'input_11')),
            #name
            EC.presence_of_element_located((By.NAME, 'input_10')),
            #address
            EC.presence_of_element_located((By.NAME, 'input_2.1')),
            #zipcode
            EC.presence_of_element_located((By.NAME, 'input_2.5')),
            #city
            EC.presence_of_element_located((By.NAME, 'input_2.3')),
            #email
            EC.presence_of_element_located((By.NAME, 'input_3')),
            #phone
            EC.presence_of_element_located((By.NAME, 'input_4'))
        )
    except Exception as e:
        print(f"An error occurred: {e}")
        continue  # Move to the next URL if an error occurs

    # Fill the form (replace with the actual form elements and data)
    driver.find_element(By.NAME, 'input_9').send_keys('Your data')
    driver.find_element(By.NAME, 'input_11').send_keys('Your data')
    driver.find_element(By.NAME, 'input_10').send_keys('Your data')
    driver.find_element(By.NAME, 'input_2.1').send_keys('Your data')
    driver.find_element(By.NAME, 'input_2.5').send_keys('Your data')
    driver.find_element(By.NAME, 'input_2.3').send_keys('Your data')
    driver.find_element(By.NAME, 'input_3').send_keys('Your data')
    driver.find_element(By.NAME, 'input_4').send_keys('Your data')

    # Submit the form (replace with the actual submit action)
    #driver.find_element(By.ID, 'gform_submit_button_1').click()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 121.0.6167.185 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0044A813+48355]
	(No symbol) [0x003DC4B1]
	(No symbol) [0x002E5358]
	(No symbol) [0x003061AC]
	(No symbol) [0x00301EF3]
	(No symbol) [0x00300579]
	(No symbol) [0x00330C55]
	(No symbol) [0x0033093C]
	(No symbol) [0x0032A536]
	(No symbol) [0x003082DC]
	(No symbol) [0x003093DD]
	GetHandleVerifier [0x006AAABD+2539405]
	GetHandleVerifier [0x006EA78F+2800735]
	GetHandleVerifier [0x006E456C+2775612]
	GetHandleVerifier [0x004D51E0+616112]
	(No symbol) [0x003E5F8C]
	(No symbol) [0x003E2328]
	(No symbol) [0x003E240B]
	(No symbol) [0x003D4FF7]
	BaseThreadInitThunk [0x76367BA9+25]
	RtlInitializeExceptionChain [0x771BBD2B+107]
	RtlClearBits [0x771BBCAF+191]
